In [3]:
import polars as pl
import polars.selectors as cs
from polars import functions as pf
import math
import os

In [1]:
# NOTE: YOU NEED HAVE RENAMED JPMORGAN IN FULL_10_Q_SCORCES MANUALY, SO DONT FORGET TO CHANGE THE NAME IN THE PARSER AND HERE
#COMPANY_NAME_LIST = ['Apple%2520Inc.%2520(AAPL)%2520(CIK%25200000320193)', 'MICROSOFT%2520CORP%2520(MSFT)%2520(CIK%25200000789019)', 
#                     'BigCommerce%2520Holdings%252C%2520Inc.%2520(BIGC)%2520(CIK%25200001626450)', 'ROKU%252C%2520INC%2520(ROKU)%2520(CIK%25200001428439)', 
#                     'JPMORGAN%2520CHASE%2520(CIK%25200000019617)', 'VISA%2520INC.%2520(V)%2520(CIK%25200001403161)', 
#                     'Block%252C%2520Inc.%2520(SQ%252C%2520BSQKZ)%2520(CIK%25200001512673)', 'Robinhood%2520Markets%252C%2520Inc.%2520(HOOD)%2520(CIK%25200001783879)', 
#                     'JOHNSON%252C%252C%2520JOHNSON%2520(JNJ)%2520(CIK%25200000200406)', 'PFIZER%2520INC%2520(PFE)%2520(CIK%25200000078003)', 
#                     'Moderna%252C%2520Inc.%2520(MRNA)%2520(CIK%25200001682852)', 'Teladoc%2520Health%252C%2520Inc.%2520(TDOC)%2520(CIK%25200001477449)', 
#                     'EXXON%2520MOBIL%2520CORP%2520(XOM)%2520(CIK%25200000034088)', 'CHEVRON%2520CORP%2520(CVX)%2520(CIK%25200000093410)', 
#                     'FIRST%2520SOLAR%252C%2520INC.%2520(FSLR)%2520(CIK%25200001274494)', 'PLUG%2520POWER%2520INC%2520(PLUG)%2520(CIK%25200001093691)', 
#                     'GENERAL%252CELECTRIC%2520CO%2520(GE)%2520(CIK%25200000040545)', '3M%2520CO%2520(MMM)%2520(CIK%25200000066740)', 
#                     'CATERPILLAR%2520INC%2520(CAT)%2520(CIK%25200000018230)', 'FASTENAL%2520CO%2520(FAST)%2520(CIK%25200000815556)']
COMPANY_NAME_LIST = ['Apple%2520Inc.%2520(AAPL)%2520(CIK%25200000320193)']

In [4]:
lm_dict = pl.read_csv(r'C:\Users\310\Desktop\Progects_Py\Parsim-sec\src\Analysis\Loughran-McDonald_MasterDictionary_1993-2021.csv')
lm_dict

Word,Seq_num,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Strong_Modal,Weak_Modal,Constraining,Syllables,Source
str,i64,i64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""AARDVARK""",1,354,1.5501e-8,1.4226e-8,0.000004,99,0,0,0,0,0,0,0,2,"""12of12inf"""
"""AARDVARKS""",2,3,1.3136e-10,8.6538e-12,9.2417e-9,1,0,0,0,0,0,0,0,2,"""12of12inf"""
"""ABACI""",3,9,3.9409e-10,1.1697e-10,5.2905e-8,7,0,0,0,0,0,0,0,3,"""12of12inf"""
"""ABACK""",4,29,1.2698e-9,6.6547e-10,1.5951e-7,28,0,0,0,0,0,0,0,2,"""12of12inf"""
"""ABACUS""",5,8570,3.7526e-7,3.8095e-7,0.000035,1108,0,0,0,0,0,0,0,3,"""12of12inf"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZYGOTE""",86529,50,2.1894e-9,8.7293e-10,1.8860e-7,35,0,0,0,0,0,0,0,2,"""12of12inf"""
"""ZYGOTES""",86530,1,4.3788e-11,1.8095e-11,1.9324e-8,1,0,0,0,0,0,0,0,2,"""12of12inf"""
"""ZYGOTIC""",86531,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,3,"""12of12inf"""


In [5]:
df_dict = {}

for company_name in COMPANY_NAME_LIST:

    clean_name = company_name.split('%')[0]
    print(clean_name)

    df_name = f"df_{clean_name}"

    file_path_partial = os.path.join(r"C:\Users\310\Desktop\Progects_Py\cleared_files", company_name)
    file_path = file_path_partial + '_reports.parquet'
    print(file_path)

    df = pl.read_parquet(file_path)

    df_dict[df_name] = df



Apple
C:\Users\310\Desktop\Progects_Py\cleared_files\Apple%2520Inc.%2520(AAPL)%2520(CIK%25200000320193)_reports.parquet


In [17]:
df_dict["df_Apple"].select(pl.col("2023-11-03").is_null().sum())

2023-11-03
u32
7985


In [6]:
total_num_of_rep = 0

for value in df_dict.values():
    total_num_of_rep += value.width
print(total_num_of_rep)

20


In [7]:
total_num_of_words = 0

for value in df_dict.values():
    for col in value:
        total_num_of_words += col.count()

print(total_num_of_words)

10371991


In [48]:
#average doc length
average_word_count = total_num_of_words/total_num_of_rep
ln_average_word_count = math.log(average_word_count)

In [9]:
#ENSURE CASE CoMpAtAbIlItY
positive_words = lm_dict.filter(lm_dict["Positive"] > 0).to_series().str.to_lowercase()

negative_words = lm_dict.filter(lm_dict["Negative"] > 0).to_series().str.to_lowercase()
print(len(negative_words) + len(positive_words))



2692


In [10]:
#example series 
example_series = df_dict['df_3M'][df_dict['df_3M'].columns[1]]
example_df = df_dict['df_3M']


This is the func, that appled to a series and get sentiment words from it. As an output we get a df with first col - grouped words that appeaps in LOUGHRAN dictionary and has >0 value in "positive" or "negative" cols. Secon col contain count of corresponding word in this document. Third col contain boolean, indicatin if this word has positive sentiment (True) or negative (False)

In [11]:
def count_sentiment_w_in_doc(df_series):
        
        #grouped series that became a df, where each row is unique word and its count in document 
        num_of_w = df_series.value_counts().rename({"count": "count in doc"})
       
        
        #first col of this df (only words by itself)
        only_w = num_of_w[num_of_w.columns[0]]
        

        #Filterd df, there is only words that have positive or negative sentiment score in this document 
        pos_w = num_of_w.filter(only_w.is_in(positive_words)).with_columns(pl.lit(True).alias("positive"))
        neg_w = num_of_w.filter(only_w.is_in(negative_words)).with_columns(pl.lit(False).alias("positive"))
       
        #Staked for one df for further operations 
        df_stacked = neg_w.vstack(pos_w)

        return df_stacked
    
check = count_sentiment_w_in_doc(df_dict['df_3M'][df_dict['df_3M'].columns[1]])  
print(check)

shape: (237, 3)
┌────────────────┬──────────────┬──────────┐
│ 2019-07-26     ┆ count in doc ┆ positive │
│ ---            ┆ ---          ┆ ---      │
│ str            ┆ u32          ┆ bool     │
╞════════════════╪══════════════╪══════════╡
│ barrier        ┆ 2            ┆ false    │
│ investigations ┆ 2            ┆ false    │
│ hazards        ┆ 2            ┆ false    │
│ question       ┆ 2            ┆ false    │
│ contend        ┆ 16           ┆ false    │
│ …              ┆ …            ┆ …        │
│ opportunities  ┆ 6            ┆ true     │
│ enjoyment      ┆ 4            ┆ true     │
│ better         ┆ 18           ┆ true     │
│ improve        ┆ 22           ┆ true     │
│ gain           ┆ 32           ┆ true     │
└────────────────┴──────────────┴──────────┘


This is the func that counts a number of docs with at least one occurense of the input word. It takes hole dictionary whith dataframes and particular word as an input. Then checks how many cols in each df contain this word, sum up all resulst and return a single value for each word.

In [12]:
def in_all_doc_occurence(check_str, df_dict):
    count = 0
    for df in df_dict.values():
        count += df.select((pl.all() == check_str).sum() > 0).sum_horizontal()[0]
        
    return count

In [13]:
stringg = "a"
in_all_doc_occurence(stringg, df_dict)

289

This is the func that takes a df from count_sentiment_w_in_doc() and for each word in this df applies in_all_doc_occurence(), meaning that it counts all docs with at least one occurence of this word. The result of applying this func for each word in df is a series of integers showing counts for correcdonding word. This series augmented af col named "count in all docs"

In [14]:
def total_occurence_count(df_series, df_dict):
   check = count_sentiment_w_in_doc(df_series).lazy()

   counts = check.with_columns(pl.col(check.columns[0]).map_elements(lambda x: in_all_doc_occurence(x, df_dict)).alias("count in all docs")).collect()
   return counts

In [94]:
res_df = total_occurence_count(example_series, df_dict)
print(res_df)

shape: (237, 4)
┌──────────────┬──────────────┬──────────┬───────────────────┐
│ 2019-07-26   ┆ count in doc ┆ positive ┆ count in all docs │
│ ---          ┆ ---          ┆ ---      ┆ ---               │
│ str          ┆ u32          ┆ bool     ┆ i64               │
╞══════════════╪══════════════╪══════════╪═══════════════════╡
│ forfeited    ┆ 2            ┆ false    ┆ 53                │
│ defendants   ┆ 74           ┆ false    ┆ 137               │
│ fraudulent   ┆ 8            ┆ false    ┆ 77                │
│ recessionary ┆ 2            ┆ false    ┆ 17                │
│ interruption ┆ 2            ┆ false    ┆ 103               │
│ …            ┆ …            ┆ …        ┆ …                 │
│ adequately   ┆ 2            ┆ true     ┆ 69                │
│ successfully ┆ 2            ┆ true     ┆ 155               │
│ gains        ┆ 58           ┆ true     ┆ 258               │
│ enhancing    ┆ 4            ┆ true     ┆ 76                │
│ successful   ┆ 2            ┆ true   

This is the func that takes as an argument resulting df, wich was obtained from total_occurence_count(). Recal that this df contains information about one document (one col in df). This func calculate weighted and raw scores using precomputed metrics. Returns a list with weighted score and raw score. Weights are calculated based on this formula: 

$$
w_{i,j} = 
\begin{cases} 
\left(\frac{1 + \log(tf_{i,j})}{1 + \log(a)}\right) \log\left(\frac{N}{df_i}\right) & \text{if } tf_{i,j} \geq 1 \\
0 & \text{otherwise} 
\end{cases}
$$


N represents the total number of 10-Ks in the sample, dfi the number of documents containing at least one occurrence of the i th word, tfi,j the raw count of the i th word in the j th document, and a the average word count in the document.

The formula is from:

Loughran T., McDonald B. When is a liability not a liability? Textual analysis, dictionaries, and 10‐Ks //The Journal of finance. – 2011. – Т. 66. – №. 1. – С. 35-65.

In [101]:
def compute_score(res_df):

    res_df.lazy()

    res = res_df.with_columns((((1 + pl.col(res_df.columns[1]).log())/(1 + ln_average_word_count))*((total_num_of_rep/pl.col(res_df.columns[3])).log())).alias("metrics"))

    w_pos_sen = res.filter(pl.col('positive') == True).select(pl.col(res.columns[4])).sum().item()
    w_neg_sen = res.filter(pl.col('positive') == False).select(pl.col(res.columns[4])).sum().item()

    w_score = w_pos_sen - w_neg_sen
    score = res.select(pl.col('positive')).sum().item()*2 - len(res)

    scores = [w_score, score]
    
    return scores



General loop that iterates over dfs and over it columns. It uses predefined finctions to calculate scores and save it as parquet files. Each parquet file named with company name is a dataframe with col names as field dates, first row is weighted scores and second row is raw scores. 

In [105]:

for df in df_dict.values():

    score_dict = {}

    for col in df:
        
        res_df = total_occurence_count(col, df_dict)
        scores = compute_score(res_df)

        score_dict[col.name] =  scores
    
    df = pl.DataFrame(score_dict)
    
    print(f'Iteration for {col[0]} is done successfully')

 # Determine the output directory and file name
    output_dir = os.path.join('.', 'full_10_Q_scores')
    os.makedirs(output_dir, exist_ok=True)

    file_name_new = f"{col[0]}_f_10_Q_score_df.parquet"
    full_path = os.path.join(output_dir, file_name_new)
    full_path = os.path.normpath(full_path)

    print(f"Attempting to write to: {full_path}")

    # Write the DataFrame to Parquet
    df.write_parquet(full_path)

    print(f"{full_path} created successfully")
        
        

Iteration for Apple%2520Inc.%2520(AAPL)%2520(CIK%25200000320193) is done successfully
Attempting to write to: full_10_Q_scores\Apple%2520Inc.%2520(AAPL)%2520(CIK%25200000320193)_f_10_Q_score_df.parquet
full_10_Q_scores\Apple%2520Inc.%2520(AAPL)%2520(CIK%25200000320193)_f_10_Q_score_df.parquet created successfully
Iteration for MICROSOFT%2520CORP%2520(MSFT)%2520(CIK%25200000789019) is done successfully
Attempting to write to: full_10_Q_scores\MICROSOFT%2520CORP%2520(MSFT)%2520(CIK%25200000789019)_f_10_Q_score_df.parquet
full_10_Q_scores\MICROSOFT%2520CORP%2520(MSFT)%2520(CIK%25200000789019)_f_10_Q_score_df.parquet created successfully
Iteration for BigCommerce%2520Holdings%252C%2520Inc.%2520(BIGC)%2520(CIK%25200001626450) is done successfully
Attempting to write to: full_10_Q_scores\BigCommerce%2520Holdings%252C%2520Inc.%2520(BIGC)%2520(CIK%25200001626450)_f_10_Q_score_df.parquet
full_10_Q_scores\BigCommerce%2520Holdings%252C%2520Inc.%2520(BIGC)%2520(CIK%25200001626450)_f_10_Q_score_df.

jupyter nbconvert --to html Parsim-sec\src\Analysis\Full_10_Q_scores_calculating.ipynb